**서포트 벡터 머신(SVM)** : 매우 강력하고 선형이나 비선형 분류, 회귀, 이상치 탐색에도 사용할 수 있는 다목적 머신러닝 모델
+ 복잡한 분류 문제에 잘 들어맞으며 작거나 중간 크기의 데이터셋에 적합

## 5.1 선형 SVM 분류

**라지 마진 분류(large margin classification)** : 클래스 사이에 가장 폭이 넓은 도로를 찾는 것

오른쪽 그래프의 실선 : SVM 분류기의 결정 경계 / 두 개의 클래스를 나누고 있을 뿐만 아니라 제일 가까운 훈련 샘플로부터 가능한 한 멀리 떨어져 있음

<img src="img/5-1.png" width="500px" align='left'>  

**서포트 벡터(support vector)** : 도로 경계에 위치한 샘플

SVM은 특성의 스케일에 민감함 -> 특성의 스케일을 조정하면 결정 경계가 훨씬 좋아짐(StandardScaler, 오른쪽 그래프)

<img src="img/5-2.png" width="500px" align='left'>  

### 5.1.1 소프트 마진 분류
**하드 마진 분류(hard margin classification)** : 모든 샘플이 도로 바깥쪽에 올바르게 분류
+ 데이터가 선형적으로 구분될 수 있어야 제대로 작동
+ 이상치에 민감함

<img src="img/5-3.png" width="500px" align='left'>  

**소프트 마진 분류(soft margin classification)** : 도로의 폭을 가능한 한 넓게 유지하는 것과 마진 오류 사이에 적절한 균형
+ 마진 오류(margin violation) : 샘플이 도로 중간이나 심지어 반대쪽에 있는 경우. 일반적으로 작은 것이 좋음
+ C(하이퍼파라미터)
    + 클 경우 : 규제가 작음 / 복잡한 모델 / 훈련 시간 긺 / 오른쪽 그래프
    + 작을 경우 : 규제가 큼 / 단순한 모델 / 훈련 시간 짧음 / 왼쪽 그래프
    + 과대적합일 경우 C를 감소시킴
    
<img src="img/5-4.png" width="500px" align='left'>  

In [4]:
# C = 1, 힌지 손실 함수를 적용한 LinearSVC 클래스 사용
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # 꽃잎 길이, 꽃잎 너비
y = (iris["target"] == 2).astype(np.float64)  # Iris virginica

svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss="hinge", random_state=42)),
    ])

svm_clf.fit(X, y)
svm_clf.predict([[5.5, 1.7]])

array([1.])

**SVC** : OvO / libsvm / 비선형 / loss = 'hinge'

**LinearSVC** : OvR / liblinear / 선형 / loss = 'squared_hinge'
+ LinearSVC -> SVC(kernel='linear', C=1)로 대체 가능
+ LinearSVC는 규제에 편향을 포함시킴. 훈련 세트에서 평균을 빼서 중앙에 맞춰야 함(StandardScaler)

## 5.2 비선형 SVM 분류

다항 특성 추가 : 비선형 데이터셋을 다루는 한 가지 방법  
$x_2 = (x_1)^2$ -> 왼쪽 데이터셋은 선형적으로 구분이 안 되지만 오른쪽은 가능

<img src="img/5-5.png" width="500px" align='left'>  

In [10]:
# PolynomialFeatures 변환기, StandardScaler, LinearSVC를 연결하여 Pipeline 생성
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15, random_state=42)
polynomial_svm_clf = Pipeline([
        ("poly_features", PolynomialFeatures(degree=3)),
        ("scaler", StandardScaler()),
        ("svm_clf", LinearSVC(C=10, loss="hinge", random_state=42))
    ])

polynomial_svm_clf.fit(X, y)

C:\Users\jwshi\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge', random_state=42))])

<img src="img/5-6.png" width="500px" align='left'>  

### 5.2.1 다항식 커널
낮은 차수의 다항식 : 매우 복잡한 데이터셋 표현 못함  
높은 차수의 다항식 : 많은 특성 탓에 모델이 느려짐

**커널 트릭(kernel trick)** : 실제로는 특성을 추가하지 않으면서 다항식 특서을 많이 추가한 것과 같은 결과를 얻을 수 있음
+ coef0 : 모델이 높은 차수와 낮은 차수에 얼마나 영향을 받을지 조절

In [15]:
from sklearn.svm import SVC

In [18]:
# 왼쪽 그래프 / 3차 다항식 커널을 사용해 SVM 분류기를 훈련
poly_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
    ])
poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

In [19]:
# 오른쪽 그래프 / 10차 다항식 커널을 사용해 SVM 분류기를 훈련
poly100_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=10, coef0=100, C=5))
    ])
poly100_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=100, degree=10, kernel='poly'))])

<img src="img/5-7.png" width="500px" align='left'>  

### 5.2.2 유사도 특성
**유사도 함수(similarity function)** : 각 샘플이 특정 랜드마크와 얼마나 닮았는지 측정

**가우시안 방사 기저 함수(radial basis function, RBF)**
+ 0(랜드마크에서 아주 멀리 떨어진 경우)부터 1(랜드마크와 같은 위치일 경우)까지 변화하며 종 모양으로 나타남
+ $\displaystyle \gamma$ : 랜드마크 지점
+ $\boldsymbol{\ell}$ : 0보다 커야 하며 값이 작을수록 폭이 넒은 종 모양

${\displaystyle \phi_{\gamma}(\mathbf{x}, \boldsymbol{\ell})} = {\displaystyle \exp({\displaystyle -\gamma \left\| \mathbf{x} - \boldsymbol{\ell} \right\|^2})}$

<img src="img/5-8.png" width="500px" align='left'>  

### 5.2.3 가우시안 RBF 커널

In [21]:
# 가우시안 RBF 커널을 사용한 SVC 모델
rbf_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
    ])
rbf_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5))])

**하이퍼파라미터 gamma($\displaystyle \gamma$)** : 규제의 역할(하이퍼파라미터 C와 비슷)
+ 증가시키면 : 좁은 종 모양 그래프. 각 샘플의 영향 범위가 작아짐
+ 감소시키면 : 넓은 종 모양 그래프. 각 샘플의 영향 범위가 커짐
+ 과대적합일 때는 감소시켜야 하고, 과소적합일 경우에는 증가시켜야 함

**여러 커널 중 언제나 선형 커널을 가장 먼저 시도해 볼 것**  
+ LinearSVC가 SVC(kernel='linear')보다 훨씬 빠름 / 특히 훈련 세트가 아주 크거나 특성 수가 많을 경우
+ 훈련 세트가 너무 크지 않다면 가우시안 RBF 커널도 좋음

<img src="img/5-9.png" width="500px" align='left'>  

### 5.2.4 계산 복잡도
**LinearSVC**
+ 선형 SVM을 위한 최적화된 알고리즘을 구현한 liblinear 라이브러리 기반

**SVC**
+ 커널 트릭 알고리즘을 구현한 libsvm 라이브러리 기반

<img src="img/표 5-1.png" width="500px" align='left'>  

## 5.3 SVM 회귀

**SVM 회귀**
+ 제한된 마진 오류 안에서 도로 안에 가능한 한 많은 샘플이 들어가도록 학습
+ 도로의 폭은 하이퍼파라미터 $\displaystyle \epsilon$ 으로 조절
+ 마진 안에서는 훈련 샘플이 추가되어도 모델의 예측에는 영향이 없음($\displaystyle \epsilon$ 에 민감하지 않다)

**SVR** : SVC의 회귀 버전. 필요한 시간이 훈련 세트에 비례해서 선형적으로 증가

**LinearSVR** : LinearSVC의 회귀 버전. 훈련 세트가 커지면 모델이 느려짐

<img src="img/5-10.png" width="500px" align='left'>  

In [22]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5, random_state=42)
svm_reg.fit(X, y)

LinearSVR(epsilon=1.5, random_state=42)

비선형 회귀 작업을 처리하기 위해 커널 SVM 모델을 사용(2차 다항 커널을 사용한 SVM 회귀)
<img src="img/5-11.png" width="500px" align='left'>

In [25]:
from sklearn.svm import SVR
svm_poly_reg1 = SVR(kernel="poly", degree=2, C=100, epsilon=0.1, gamma="scale")
svm_poly_reg1.fit(X, y)

SVR(C=100, degree=2, kernel='poly')

In [24]:
svm_poly_reg2 = SVR(kernel="poly", degree=2, C=0.01, epsilon=0.1, gamma="scale")
svm_poly_reg2.fit(X, y)

SVR(C=0.01, degree=2, kernel='poly')

## 5.4 SVM 이론